In [2]:
# signal_logic.py

import importlib
import os
import sys

import pandas as pd
import yaml

# --- Add project root to path ---
project_root = os.path.abspath(".")
if project_root not in sys.path:
    sys.path.append(project_root)

# --- Load config ---
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)


# --- Dynamic function loader ---
def load_func(module_path, func_name):
    module = importlib.import_module(module_path)
    return getattr(module, func_name)


# --- Load confirmation indicator functions (C1 and C2 from same file) ---
c1_func = load_func("indicators.confirmation_funcs", f"c1_{config['indicators']['c1']}")

c2_func = None
if config["indicators"].get("use_c2"):
    c2_func = load_func("indicators.confirmation_funcs", f"c1_{config['indicators']['c2']}")

# --- Load baseline function ---
baseline_func = None
if config["indicators"].get("use_baseline"):
    baseline_func = load_func(
        "indicators.baseline_funcs", f"baseline_{config['indicators']['baseline']}"
    )

# --- Load volume function ---
volume_func = None
if config["indicators"].get("use_volume"):
    volume_func = load_func("indicators.volume_funcs", f"volume_{config['indicators']['volume']}")

# --- Load exit function ---
exit_func = None
if config["indicators"].get("use_exit"):
    exit_func = load_func("indicators.exit_funcs", f"exit_{config['indicators']['exit']}")

FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'

In [ ]:
# Load your data
df = pd.read_csv("data/daily/EURUSD.csv")

# Apply selected indicators
df = c1_func(df)
if c2_func:
    df = c2_func(df)
if baseline_func:
    df = baseline_func(df)
if volume_func:
    df = volume_func(df)
if exit_func:
    df = exit_func(df)

In [ ]:
# --- Entry + Filter Rules ---
def apply_signal_logic(df):
    """
    Applies full entry and exit logic using:
    - C1 or baseline as trigger
    - One Candle Rule
    - Pullback Rule
    - All filters must align within one candle
    """
    df["entry_signal"] = 0
    df["exit_signal"] = 0

    pending_trade = None  # store C1 signal when filters fail

    for i in range(1, len(df)):
        # === EXIT LOGIC ===
        c1 = df.loc[i, "c1_signal"]
        close = df.loc[i, "Close"]
        base_val = df.loc[i, "baseline"] if "baseline" in df.columns else None
        atr = df.loc[i, "ATR"]

        # Exit if C1 reverses
        if c1 == -1 * df.loc[i - 1, "c1_signal"]:
            df.loc[i, "exit_signal"] = 1

        # Exit if price crosses baseline
        if baseline_func:
            prev_close = df.loc[i - 1, "Close"]
            prev_base = df.loc[i - 1, "baseline"]
            crossed = (close > base_val and prev_close < prev_base) or (
                close < base_val and prev_close > prev_base
            )
            if crossed:
                df.loc[i, "exit_signal"] = 1

        # Exit if exit indicator fires
        if exit_func and df.loc[i, "exit_signal"] == 0:
            if df.loc[i, "exit_signal"] == 1:
                df.loc[i, "exit_signal"] = 1

        # === ENTRY LOGIC ===
        direction = 0
        USE_BASELINE_TRIGGER = config["rules"].get("baseline_as_trigger", False)

        # Handle pending trades from previous candle
        if pending_trade and i == pending_trade["index"] + 1:
            recovered = all(
                did_filter_pass(df, i, f, pending_trade["direction"], atr)
                for f in pending_trade["failed_filters"]
            )
            if recovered:
                df.loc[i, "entry_signal"] = pending_trade["direction"]
            pending_trade = None

        # C1 as trigger
        elif c1 != 0:
            direction = c1
            passed, failed_filters = get_filter_status(df, i, direction, atr)
            if passed:
                df.loc[i, "entry_signal"] = direction
            elif config["rules"].get("one_candle_rule") or config["rules"].get("pullback_rule"):
                pending_trade = {
                    "index": i,
                    "direction": direction,
                    "failed_filters": failed_filters,
                }

        # Baseline as trigger
        elif USE_BASELINE_TRIGGER and baseline_func:
            direction = 1 if close > base_val else -1
            passed, failed_filters = get_filter_status(df, i, direction, atr)

            # Bridge Too Far rule
            last_c1_index = None
            for j in range(i - 1, max(0, i - 10), -1):
                if df.loc[j, "c1_signal"] != 0:
                    last_c1_index = j
                    break
            days_since_c1 = i - last_c1_index if last_c1_index is not None else 999

            if passed and days_since_c1 < 7:
                df.loc[i, "entry_signal"] = direction
            elif config["rules"].get("one_candle_rule") or config["rules"].get("pullback_rule"):
                pending_trade = {
                    "index": i,
                    "direction": direction,
                    "failed_filters": failed_filters,
                }

    return df

In [ ]:
def get_filter_status(df, i, direction, atr):
    """
    Checks which filters pass or fail on candle i
    Returns (passed: bool, failed_filters: list)
    """
    failed = []
    USE_C2 = config["indicators"].get("use_c2", False)
    USE_VOLUME = config["indicators"].get("use_volume", False)
    USE_PULLBACK = config["rules"].get("pullback_rule", False)

    # --- C2 ---
    if USE_C2:
        if df.loc[i, "c2_signal"] != direction:
            failed.append("c2")

    # --- Volume ---
    if USE_VOLUME:
        if df.loc[i, "volume_signal"] != 1:
            failed.append("volume")

    # --- Baseline direction ---
    if baseline_func:
        price = df.loc[i, "Close"]
        base = df.loc[i, "baseline"]
        if not ((direction == 1 and price > base) or (direction == -1 and price < base)):
            failed.append("baseline_dir")

        # --- Pullback Rule ---
        if USE_PULLBACK:
            too_far = (direction == 1 and price > base + atr) or (
                direction == -1 and price < base - atr
            )
            prev_price = df.loc[i - 1, "Close"]
            prev_too_far = (direction == 1 and prev_price > base + atr) or (
                direction == -1 and prev_price < base - atr
            )
            if too_far and prev_too_far:
                failed.append("pullback")

    return (len(failed) == 0), failed


def did_filter_pass(df, i, filter_name, direction, atr):
    """
    On Day 2, check if each previously failed filter passed
    """
    if filter_name == "c2":
        return df.loc[i, "c2_signal"] == direction
    elif filter_name == "volume":
        return df.loc[i, "volume_signal"] == 1
    elif filter_name == "baseline_dir":
        base = df.loc[i, "baseline"]
        price = df.loc[i, "Close"]
        return (direction == 1 and price > base) or (direction == -1 and price < base)
    elif filter_name == "pullback":
        base = df.loc[i, "baseline"]
        price = df.loc[i, "Close"]
        return not (
            (direction == 1 and price > base + atr) or (direction == -1 and price < base - atr)
        )
    return True  # fallback

In [ ]:
# --- Helper: Check all filters and rules ---
def check_filters(df, i, direction, atr, baseline_func, vol, c2_func, catalyst):
    USE_C2 = config["indicators"].get("use_c2", False)
    USE_VOLUME = config["indicators"].get("use_volume", False)
    USE_ONE = config["rules"].get("one_candle_rule", False)
    USE_PULLBACK = config["rules"].get("pullback_rule", False)

    # --- C2 Filter ---
    if USE_C2:
        c2_today = df.loc[i, "c2_signal"]
        c2_prev = df.loc[i - 1, "c2_signal"]
        if not (c2_today == direction or (USE_ONE and c2_prev == direction)):
            return False

    # --- Volume Filter ---
    if USE_VOLUME:
        v_today = df.loc[i, "volume_signal"]
        v_prev = df.loc[i - 1, "volume_signal"]
        if not (v_today == 1 or (USE_ONE and v_prev == 1)):
            return False

    # --- Baseline Filter ---
    if baseline_func:
        price = df.loc[i, "Close"]
        base = df.loc[i, "baseline"]
        if not ((direction == 1 and price > base) or (direction == -1 and price < base)):
            return False

        if USE_PULLBACK:
            too_far = (direction == 1 and price > base + atr) or (
                direction == -1 and price < base - atr
            )
            prev_price = df.loc[i - 1, "Close"]
            prev_too_far = (direction == 1 and prev_price > base + atr) or (
                direction == -1 and prev_price < base - atr
            )
            if too_far and prev_too_far:
                return False

    return True